In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import pmdarima as pm
from datetime import datetime, timedelta

In [21]:
lst = []
f = open('news(en).csv')
i = 0
for line in f:
    if i > 0:
        contents = line.split(',')
        company = contents[0]
        date = contents[1]
        title = contents[2]
        text = ' '.join(contents[3:])
        text = text.replace(',',' ')
        text = text.strip()
        lst.append([company, date, title, text])
    i += 1

In [22]:
df = pd.DataFrame(lst, columns =['company', 'date','title','text']) 

In [23]:
df

,company,date,title,text
0,intel,2018-01-03,Intel's alleged security flaw could cost chipm...,Bernstein says A report alleging Intel chips h...
1,tesla,2018-01-03,Tesla delivers 1,550 Model 3 sedans and 29 870 total vehicles i...
2,apple,2018-01-02,Data Sheet—An Apple Conspiracy Theory Blooms,Good morning and happy new year. Aaron in for ...
3,google,2018-01-03,Amazon Echo vs. Google Home: How Competing Pri...,Amazon.com and Alphabet ‘s Google both discoun...
4,amazon,2018-01-03,Amazon Echo vs. Google Home: How Competing Pri...,Amazon.com and Alphabet ‘s Google both discoun...
5,google,2018-01-03,Amazon,Google cut speaker prices in market share cont...
6,amazon,2018-01-03,Amazon,Google cut speaker prices in market share cont...
7,amazon,2018-01-03,Here's why Amazon may not buy Target,Here's why Amazon may not buy Target Loup Vent...
8,intel,2018-01-03,Intel CEO: We believe we have the right fixes ...,Intel CEO Brian Krzanich said researchers at G...
9,apple,2018-01-03,Brazilian agency requires Apple to inform cons...,Brazilian agency requires Apple to inform cons...


In [24]:
msk = np.random.rand(len(df)) < 0.8
traindata = df[msk]
testdata = df[~msk]

In [25]:
apple = pd.read_csv('data/AAPL.csv')
amazon = pd.read_csv('data/AMZN.csv')
facebook = pd.read_csv('data/FB.csv')
google = pd.read_csv('data/GOOGL.csv')
microsoft = pd.read_csv('data/MSFT.csv')
tesla = pd.read_csv('data/TSLA.csv')
intel = pd.read_csv('data/INTC.csv')

In [26]:
testdata['date']=pd.to_datetime(testdata['date'])
testdata=testdata.sort_values('date')
testdata = testdata.reset_index(drop=True)
traindata['date']=pd.to_datetime(traindata['date'])
traindata=traindata.sort_values('date')
traindata = traindata.reset_index(drop=True)

/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [28]:
traindata

,company,date,title,text
0,tesla,2017-12-27,Tesla to make pickup truck after Model Y cross...,Wed Dec 27 2017 / 4:49 PM EST Tesla to make p...
1,google,2018-01-01,Former Google career coach shares ways to ace ...,Getty Images SHARES Starting a new job can fe...
2,amazon,2018-01-01,The Limits of Amazon,Amazon.com Inc. is a colossus. In the near fut...
3,amazon,2018-01-01,Toyota Working With Amazon,Uber to Develop Self-Driving Shuttle | Fortune...
4,amazon,2018-01-02,BRIEF-Amazon.Com Says ‍More Than Five Bln Item...,34 PM / Updated 8 minutes ago BRIEF-Amazon.Com...
5,facebook,2018-01-02,Why Shares of Facebook,Amazon Netflix and Google Can Rise Further i...
6,amazon,2018-01-02,Why Shares of Facebook,Amazon Netflix and Google Can Rise Further i...
7,google,2018-01-02,Why Shares of Facebook,Amazon Netflix and Google Can Rise Further i...
8,apple,2018-01-02,CLSA: Apple iPhone X expectations 'still too h...,Apple iPhone X volume expectations are still t...
9,apple,2018-01-02,Sell Apple on battery life canard? No!: Jim Cr...,Sell Apple on battery life canard? No!: Jim Cr...


In [48]:
def getdate(timestr):
    date = timestr.split(' ')[0]
    year = '20'+date.split('/')[2]
    month = '0'+date.split('/')[0]
    month = month[-2:]
    day = '0'+date.split('/')[1]
    day = day[-2:]
    return year+'-'+month+'-'+day
    

In [49]:
apple['date']=pd.to_datetime(apple['datetime'].apply(getdate))
apple['date']=pd.to_datetime(apple['date'])
apple=apple.sort_values('date')
apple = apple.reset_index(drop=True)

amazon['date']=pd.to_datetime(amazon['datetime'].apply(getdate))
amazon['date']=pd.to_datetime(amazon['date'])
amazon=amazon.sort_values('date')
amazon = amazon.reset_index(drop=True)

facebook['date']=pd.to_datetime(facebook['datetime'].apply(getdate))
facebook['date']=pd.to_datetime(facebook['date'])
facebook=facebook.sort_values('date')
facebook = facebook.reset_index(drop=True)

google['date']=pd.to_datetime(google['datetime'].apply(getdate))
google['date']=pd.to_datetime(google['date'])
google=google.sort_values('date')
google = google.reset_index(drop=True)

microsoft['date']=pd.to_datetime(microsoft['datetime'].apply(getdate))
microsoft['date']=pd.to_datetime(microsoft['date'])
microsoft=microsoft.sort_values('date')
microsoft = microsoft.reset_index(drop=True)

tesla['date']=pd.to_datetime(tesla['datetime'].apply(getdate))
tesla['date']=pd.to_datetime(tesla['date'])
tesla=tesla.sort_values('date')
tesla = tesla.reset_index(drop=True)

intel['date']=pd.to_datetime(intel['datetime'].apply(getdate))
intel['date']=pd.to_datetime(intel['date'])
intel=intel.sort_values('date')
intel = intel.reset_index(drop=True)

In [39]:
testdata

,company,date,title,text
0,apple,2017-12-27,Apple Watch well being options reveal new fron...,Author Natalia Camp Published December 27 201...
1,google,2018-01-01,A Browser You’ve Never Heard of Is Dethroning ...,JAKARTA Indonesia—A mobile browser rarely use...
2,apple,2018-01-02,We see Apple doing more 'add-on' acquisitions:...,We see Apple doing more 'add-on' acquisitions:...
3,intel,2018-01-02,Inspirata Acquires Toronto-based Artificial In...,Inc. (AIM) to Enhance its R&D in Artificial In...
4,google,2018-01-02,The Wall Street Journal: The Alibaba browser n...,Published: Jan 2 2018 8:32 a.m. ET Share UC ...
5,amazon,2018-01-02,Amazon will buy Target in 2018,influential tech analyst Gene Munster predicts...
6,google,2018-01-03,Amazon and Google cut speaker prices in market...,analysts say Amazon.com and Alphabet's Google ...
7,facebook,2018-01-03,Facebook to Announce Fourth Quarter and Full Y...,MENLO PARK Calif. Jan. 2 2018 /PRNewswire/ ...
8,amazon,2018-01-03,Amazon grabbed 4 percent of all US retail sale...,There's no denying that Amazon 's retail busin...
9,google,2018-01-03,Google discovers 'serious' flaws in Intel and ...,SANTA CLARA Calif. (AP) — Security researcher...


In [50]:
apple

,datetime,open,high,low,close,volume,date
0,11/1/17 5:00,169.870,169.9400,165.6100,166.89,33637762,2017-11-01
1,11/2/17 5:00,167.640,168.5000,165.2800,168.11,41393373,2017-11-02
2,11/3/17 5:00,174.000,174.2600,171.1200,172.50,59398631,2017-11-03
3,11/6/17 6:00,172.365,174.9900,171.7200,174.25,35026306,2017-11-06
4,11/7/17 6:00,173.910,175.2500,173.6000,174.81,24361485,2017-11-07
5,11/8/17 6:00,174.660,176.2400,174.3300,176.24,24409527,2017-11-08
6,11/9/17 6:00,175.110,176.0950,173.1400,175.88,29482596,2017-11-09
7,11/10/17 6:00,175.110,175.3800,174.2700,174.67,25145500,2017-11-10
8,11/13/17 6:00,173.500,174.5000,173.4000,173.97,16982080,2017-11-13
9,11/14/17 6:00,173.040,173.4800,171.1800,171.34,24782487,2017-11-14


In [68]:
stockdf = intel.copy()
print(type(stockdf))
stockdf = apple.copy()
print(type(stockdf))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [70]:
for i in range(len(traindata)):
    print(i)
    newsdate = traindata.loc[i, "date"]
    company = traindata.loc[i, "company"]
    if company == 'apple':
        stockdf = apple.copy()
    if company == 'amazon':
        stockdf = amazon.copy()
    if company == 'facebook':
        stockdf = facebook.copy()
    if company == 'google':
        stockdf = google.copy()
    if company == 'microsoft':
        stockdf = microsoft.copy()
    if company == 'tesla':
        stockdf = tesla.copy()
    if company == 'intel':
        stockdf = intel.copy()
    print(type(stockdf))
    stockdf = stockdf[['date','close']]   
    test = stockdf[stockdf['date'] >= newsdate]
    test = test.reset_index(drop=True)
    if len(test) > 0:
        nextclose = test.loc[0, "close"]
        train = stockdf[stockdf['date'] < newsdate]
        if len(train) > 0:
            train = train[-min(30,len(train)):]        
            train = train.set_index('date',drop=True)
            train = train['close']
            train.index = pd.DatetimeIndex(train.index).to_period('D')
            lastclose=train[-1]
            traindata.loc[i,'nextclose'] = nextclose
            traindata.loc[i,'lastclose'] = lastclose

0
<class 'pandas.core.frame.DataFrame'>
1
<class 'pandas.core.frame.DataFrame'>
2
<class 'pandas.core.frame.DataFrame'>
3
<class 'pandas.core.frame.DataFrame'>
4
<class 'pandas.core.frame.DataFrame'>
5
<class 'pandas.core.frame.DataFrame'>
6
<class 'pandas.core.frame.DataFrame'>
7
<class 'pandas.core.frame.DataFrame'>
8
<class 'pandas.core.frame.DataFrame'>
9
<class 'pandas.core.frame.DataFrame'>
10
<class 'pandas.core.frame.DataFrame'>
11
<class 'pandas.core.frame.DataFrame'>
12
<class 'pandas.core.frame.DataFrame'>
13
<class 'pandas.core.frame.DataFrame'>
14
<class 'pandas.core.frame.DataFrame'>
15
<class 'pandas.core.frame.DataFrame'>
16
<class 'pandas.core.frame.DataFrame'>
17
<class 'pandas.core.frame.DataFrame'>
18
<class 'pandas.core.frame.DataFrame'>
19
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
21
<class 'pandas.core.frame.DataFrame'>
22
<class 'pandas.core.frame.DataFrame'>
23
<class 'pandas.core.frame.DataFrame'>
24
<class 'pandas.core.fra

201
<class 'pandas.core.frame.DataFrame'>
202
<class 'pandas.core.frame.DataFrame'>
203
<class 'pandas.core.frame.DataFrame'>
204
<class 'pandas.core.frame.DataFrame'>
205
<class 'pandas.core.frame.DataFrame'>
206
<class 'pandas.core.frame.DataFrame'>
207
<class 'pandas.core.frame.DataFrame'>
208
<class 'pandas.core.frame.DataFrame'>
209
<class 'pandas.core.frame.DataFrame'>
210
<class 'pandas.core.frame.DataFrame'>
211
<class 'pandas.core.frame.DataFrame'>
212
<class 'pandas.core.frame.DataFrame'>
213
<class 'pandas.core.frame.DataFrame'>
214
<class 'pandas.core.frame.DataFrame'>
215
<class 'pandas.core.frame.DataFrame'>
216
<class 'pandas.core.frame.DataFrame'>
217
<class 'pandas.core.frame.DataFrame'>
218
<class 'pandas.core.frame.DataFrame'>
219
<class 'pandas.core.frame.DataFrame'>
220
<class 'pandas.core.frame.DataFrame'>
221
<class 'pandas.core.frame.DataFrame'>
222
<class 'pandas.core.frame.DataFrame'>
223
<class 'pandas.core.frame.DataFrame'>
224
<class 'pandas.core.frame.Data

436
<class 'pandas.core.frame.DataFrame'>
437
<class 'pandas.core.frame.DataFrame'>
438
<class 'pandas.core.frame.DataFrame'>
439
<class 'pandas.core.frame.DataFrame'>
440
<class 'pandas.core.frame.DataFrame'>
441
<class 'pandas.core.frame.DataFrame'>
442
<class 'pandas.core.frame.DataFrame'>
443
<class 'pandas.core.frame.DataFrame'>
444
<class 'pandas.core.frame.DataFrame'>
445
<class 'pandas.core.frame.DataFrame'>
446
<class 'pandas.core.frame.DataFrame'>
447
<class 'pandas.core.frame.DataFrame'>
448
<class 'pandas.core.frame.DataFrame'>
449
<class 'pandas.core.frame.DataFrame'>
450
<class 'pandas.core.frame.DataFrame'>
451
<class 'pandas.core.frame.DataFrame'>
452
<class 'pandas.core.frame.DataFrame'>
453
<class 'pandas.core.frame.DataFrame'>
454
<class 'pandas.core.frame.DataFrame'>
455
<class 'pandas.core.frame.DataFrame'>
456
<class 'pandas.core.frame.DataFrame'>
457
<class 'pandas.core.frame.DataFrame'>
458
<class 'pandas.core.frame.DataFrame'>
459
<class 'pandas.core.frame.Data

663
<class 'pandas.core.frame.DataFrame'>
664
<class 'pandas.core.frame.DataFrame'>
665
<class 'pandas.core.frame.DataFrame'>
666
<class 'pandas.core.frame.DataFrame'>
667
<class 'pandas.core.frame.DataFrame'>
668
<class 'pandas.core.frame.DataFrame'>
669
<class 'pandas.core.frame.DataFrame'>
670
<class 'pandas.core.frame.DataFrame'>
671
<class 'pandas.core.frame.DataFrame'>
672
<class 'pandas.core.frame.DataFrame'>
673
<class 'pandas.core.frame.DataFrame'>
674
<class 'pandas.core.frame.DataFrame'>
675
<class 'pandas.core.frame.DataFrame'>
676
<class 'pandas.core.frame.DataFrame'>
677
<class 'pandas.core.frame.DataFrame'>
678
<class 'pandas.core.frame.DataFrame'>
679
<class 'pandas.core.frame.DataFrame'>
680
<class 'pandas.core.frame.DataFrame'>
681
<class 'pandas.core.frame.DataFrame'>
682
<class 'pandas.core.frame.DataFrame'>
683
<class 'pandas.core.frame.DataFrame'>
684
<class 'pandas.core.frame.DataFrame'>
685
<class 'pandas.core.frame.DataFrame'>
686
<class 'pandas.core.frame.Data

875
<class 'pandas.core.frame.DataFrame'>
876
<class 'pandas.core.frame.DataFrame'>
877
<class 'pandas.core.frame.DataFrame'>
878
<class 'pandas.core.frame.DataFrame'>
879
<class 'pandas.core.frame.DataFrame'>
880
<class 'pandas.core.frame.DataFrame'>
881
<class 'pandas.core.frame.DataFrame'>
882
<class 'pandas.core.frame.DataFrame'>
883
<class 'pandas.core.frame.DataFrame'>
884
<class 'pandas.core.frame.DataFrame'>
885
<class 'pandas.core.frame.DataFrame'>
886
<class 'pandas.core.frame.DataFrame'>
887
<class 'pandas.core.frame.DataFrame'>
888
<class 'pandas.core.frame.DataFrame'>
889
<class 'pandas.core.frame.DataFrame'>
890
<class 'pandas.core.frame.DataFrame'>
891
<class 'pandas.core.frame.DataFrame'>
892
<class 'pandas.core.frame.DataFrame'>
893
<class 'pandas.core.frame.DataFrame'>
894
<class 'pandas.core.frame.DataFrame'>
895
<class 'pandas.core.frame.DataFrame'>
896
<class 'pandas.core.frame.DataFrame'>
897
<class 'pandas.core.frame.DataFrame'>
898
<class 'pandas.core.frame.Data

...

6875
<class 'pandas.core.frame.DataFrame'>
6876
<class 'pandas.core.frame.DataFrame'>
6877
<class 'pandas.core.frame.DataFrame'>
6878
<class 'pandas.core.frame.DataFrame'>
6879
<class 'pandas.core.frame.DataFrame'>
6880
<class 'pandas.core.frame.DataFrame'>
6881
<class 'pandas.core.frame.DataFrame'>
6882
<class 'pandas.core.frame.DataFrame'>
6883
<class 'pandas.core.frame.DataFrame'>
6884
<class 'pandas.core.frame.DataFrame'>
6885
<class 'pandas.core.frame.DataFrame'>
6886
<class 'pandas.core.frame.DataFrame'>
6887
<class 'pandas.core.frame.DataFrame'>
6888
<class 'pandas.core.frame.DataFrame'>
6889
<class 'pandas.core.frame.DataFrame'>
6890
<class 'pandas.core.frame.DataFrame'>
6891
<class 'pandas.core.frame.DataFrame'>
6892
<class 'pandas.core.frame.DataFrame'>
6893
<class 'pandas.core.frame.DataFrame'>
6894
<class 'pandas.core.frame.DataFrame'>
6895
<class 'pandas.core.frame.DataFrame'>
6896
<class 'pandas.core.frame.DataFrame'>
6897
<class 'pandas.core.frame.DataFrame'>
6898
<class

7081
<class 'pandas.core.frame.DataFrame'>
7082
<class 'pandas.core.frame.DataFrame'>
7083
<class 'pandas.core.frame.DataFrame'>
7084
<class 'pandas.core.frame.DataFrame'>
7085
<class 'pandas.core.frame.DataFrame'>
7086
<class 'pandas.core.frame.DataFrame'>
7087
<class 'pandas.core.frame.DataFrame'>
7088
<class 'pandas.core.frame.DataFrame'>
7089
<class 'pandas.core.frame.DataFrame'>
7090
<class 'pandas.core.frame.DataFrame'>
7091
<class 'pandas.core.frame.DataFrame'>
7092
<class 'pandas.core.frame.DataFrame'>
7093
<class 'pandas.core.frame.DataFrame'>
7094
<class 'pandas.core.frame.DataFrame'>
7095
<class 'pandas.core.frame.DataFrame'>
7096
<class 'pandas.core.frame.DataFrame'>
7097
<class 'pandas.core.frame.DataFrame'>
7098
<class 'pandas.core.frame.DataFrame'>
7099
<class 'pandas.core.frame.DataFrame'>
7100
<class 'pandas.core.frame.DataFrame'>
7101
<class 'pandas.core.frame.DataFrame'>
7102
<class 'pandas.core.frame.DataFrame'>
7103
<class 'pandas.core.frame.DataFrame'>
7104
<class

7288
<class 'pandas.core.frame.DataFrame'>
7289
<class 'pandas.core.frame.DataFrame'>
7290
<class 'pandas.core.frame.DataFrame'>
7291
<class 'pandas.core.frame.DataFrame'>
7292
<class 'pandas.core.frame.DataFrame'>
7293
<class 'pandas.core.frame.DataFrame'>
7294
<class 'pandas.core.frame.DataFrame'>
7295
<class 'pandas.core.frame.DataFrame'>
7296
<class 'pandas.core.frame.DataFrame'>
7297
<class 'pandas.core.frame.DataFrame'>
7298
<class 'pandas.core.frame.DataFrame'>
7299
<class 'pandas.core.frame.DataFrame'>
7300
<class 'pandas.core.frame.DataFrame'>
7301
<class 'pandas.core.frame.DataFrame'>
7302
<class 'pandas.core.frame.DataFrame'>
7303
<class 'pandas.core.frame.DataFrame'>
7304
<class 'pandas.core.frame.DataFrame'>
7305
<class 'pandas.core.frame.DataFrame'>
7306
<class 'pandas.core.frame.DataFrame'>
7307
<class 'pandas.core.frame.DataFrame'>
7308
<class 'pandas.core.frame.DataFrame'>
7309
<class 'pandas.core.frame.DataFrame'>
7310
<class 'pandas.core.frame.DataFrame'>
7311
<class

7485
<class 'pandas.core.frame.DataFrame'>
7486
<class 'pandas.core.frame.DataFrame'>
7487
<class 'pandas.core.frame.DataFrame'>
7488
<class 'pandas.core.frame.DataFrame'>
7489
<class 'pandas.core.frame.DataFrame'>
7490
<class 'pandas.core.frame.DataFrame'>
7491
<class 'pandas.core.frame.DataFrame'>
7492
<class 'pandas.core.frame.DataFrame'>
7493
<class 'pandas.core.frame.DataFrame'>
7494
<class 'pandas.core.frame.DataFrame'>
7495
<class 'pandas.core.frame.DataFrame'>
7496
<class 'pandas.core.frame.DataFrame'>
7497
<class 'pandas.core.frame.DataFrame'>
7498
<class 'pandas.core.frame.DataFrame'>
7499
<class 'pandas.core.frame.DataFrame'>
7500
<class 'pandas.core.frame.DataFrame'>
7501
<class 'pandas.core.frame.DataFrame'>
7502
<class 'pandas.core.frame.DataFrame'>
7503
<class 'pandas.core.frame.DataFrame'>
7504
<class 'pandas.core.frame.DataFrame'>
7505
<class 'pandas.core.frame.DataFrame'>
7506
<class 'pandas.core.frame.DataFrame'>
7507
<class 'pandas.core.frame.DataFrame'>
7508
<class

7682
<class 'pandas.core.frame.DataFrame'>
7683
<class 'pandas.core.frame.DataFrame'>
7684
<class 'pandas.core.frame.DataFrame'>
7685
<class 'pandas.core.frame.DataFrame'>
7686
<class 'pandas.core.frame.DataFrame'>
7687
<class 'pandas.core.frame.DataFrame'>
7688
<class 'pandas.core.frame.DataFrame'>
7689
<class 'pandas.core.frame.DataFrame'>
7690
<class 'pandas.core.frame.DataFrame'>
7691
<class 'pandas.core.frame.DataFrame'>
7692
<class 'pandas.core.frame.DataFrame'>
7693
<class 'pandas.core.frame.DataFrame'>
7694
<class 'pandas.core.frame.DataFrame'>
7695
<class 'pandas.core.frame.DataFrame'>
7696
<class 'pandas.core.frame.DataFrame'>
7697
<class 'pandas.core.frame.DataFrame'>
7698
<class 'pandas.core.frame.DataFrame'>
7699
<class 'pandas.core.frame.DataFrame'>
7700
<class 'pandas.core.frame.DataFrame'>
7701
<class 'pandas.core.frame.DataFrame'>
7702
<class 'pandas.core.frame.DataFrame'>
7703
<class 'pandas.core.frame.DataFrame'>
7704
<class 'pandas.core.frame.DataFrame'>
7705
<class

In [77]:
for i in range(len(testdata)):
    print(i)
    newsdate = testdata.loc[i, "date"]
    company = traindata.loc[i, "company"]
    if company == 'apple':
        stockdf = apple.copy()
    if company == 'amazon':
        stockdf = amazon.copy()
    if company == 'facebook':
        stockdf = facebook.copy()
    if company == 'google':
        stockdf = google.copy()
    if company == 'microsoft':
        stockdf = microsoft.copy()
    if company == 'tesla':
        stockdf = tesla.copy()
    if company == 'intel':
        stockdf = intel.copy()
    stockdf = stockdf[['date','close']]
    test = stockdf[stockdf['date'] >= newsdate]
    test = test.reset_index(drop=True)
    if len(test) > 0:
        nextclose = test.loc[0, "close"]
        train = stockdf[stockdf['date'] < newsdate]
        if len(train) > 0:
            train = train[-min(30,len(train)):]        
            train = train.set_index('date',drop=True)
            train = train['close']
            train.index = pd.DatetimeIndex(train.index).to_period('D')
            try:
                p = min(5, len(train))
                d = 1
                q = 1
                model = ARIMA(train, order=(p,d,q))
                model_fit = model.fit(disp=False)
                forecast, stderr, conf = model_fit.forecast()
                lastclose=train[-1]
                timeseriesforecast=forecast
                timeserieslower = conf[0][0]
                timeseriesupper = conf[0][1]
                testdata.loc[i,'nextclose'] = nextclose
                testdata.loc[i,'lastclose'] = lastclose
                testdata.loc[i,'timeseriesforecast'] = timeseriesforecast
                testdata.loc[i,'timeserieslower'] = timeserieslower
                testdata.loc[i,'timeseriesupper'] = timeseriesupper

            except:
                lastclose=train[-1]
                testdata.loc[i,'nextclose'] = nextclose
                testdata.loc[i,'lastclose'] = lastclose


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37


/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


38
39


/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


40


...

1781
1782
1783
1784
1785
1786
1787
1788
1789
1790
1791
1792
1793
1794
1795


/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


1796
1797
1798
1799
1800
1801
1802
1803
1804


/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


1805
1806
1807
1808
1809
1810
1811
1812
1813
1814
1815
1816
1817
1818
1819
1820
1821
1822
1823
1824
1825
1826
1827
1828
1829
1830
1831
1832
1833
1834
1835
1836
1837
1838
1839
1840
1841


In [4]:
stock['id'].nunique()

3080

In [5]:
traindata['code'].nunique()

3530

In [79]:
testdata1 = testdata[['date','company','title','text','nextclose','lastclose','timeseriesforecast','timeserieslower','timeseriesupper']]


In [71]:
traindata1 = traindata[['date','company','title','text','nextclose','lastclose']]

In [80]:
testdata1 = testdata1[testdata1['nextclose'] > 0]

In [83]:
testdata1.to_csv('data/tefinal(EN)0414.csv',index=False)

In [81]:
testdata1.loc[testdata1['nextclose']-testdata1['lastclose'] >= 0, 'groundtruth01'] = int(1)
testdata1.loc[testdata1['nextclose']-testdata1['lastclose'] < 0, 'groundtruth01'] = int(0)

In [82]:
testdata1.loc[testdata1['timeseriesforecast']-testdata1['lastclose'] >= 0, 'timeseries01'] = int(1)
testdata1.loc[testdata1['timeseriesforecast']-testdata1['lastclose'] < 0, 'timeseries01'] = int(0)

In [20]:
traindata1[traindata1.isnull().any(axis=1)]['code'].nunique()

998

In [84]:
testdata1

,date,company,title,text,nextclose,lastclose,timeseriesforecast,timeserieslower,timeseriesupper,groundtruth01,timeseries01
0,2017-12-27,apple,Apple Watch well being options reveal new fron...,Author Natalia Camp Published December 27 201...,311.64,317.29,321.076679,313.307849,328.845508,0.0,1.0
1,2018-01-01,google,A Browser You’ve Never Heard of Is Dethroning ...,JAKARTA Indonesia—A mobile browser rarely use...,1073.21,1053.40,1053.122599,1039.585449,1066.659749,1.0,0.0
2,2018-01-02,apple,We see Apple doing more 'add-on' acquisitions:...,We see Apple doing more 'add-on' acquisitions:...,1189.01,1169.47,1173.742521,1153.901074,1193.583968,1.0,1.0
3,2018-01-02,intel,Inspirata Acquires Toronto-based Artificial In...,Inc. (AIM) to Enhance its R&D in Artificial In...,1189.01,1169.47,1173.742521,1153.901074,1193.583968,1.0,1.0
4,2018-01-02,google,The Wall Street Journal: The Alibaba browser n...,Published: Jan 2 2018 8:32 a.m. ET Share UC ...,1189.01,1169.47,1173.742521,1153.901074,1193.583968,1.0,1.0
5,2018-01-02,amazon,Amazon will buy Target in 2018,influential tech analyst Gene Munster predicts...,181.42,176.46,176.314252,172.420451,180.208052,1.0,0.0
6,2018-01-03,google,Amazon and Google cut speaker prices in market...,analysts say Amazon.com and Alphabet's Google ...,1204.20,1189.01,1189.709884,1169.323951,1210.095818,1.0,1.0
7,2018-01-03,facebook,Facebook to Announce Fourth Quarter and Full Y...,MENLO PARK Calif. Jan. 2 2018 /PRNewswire/ ...,1091.52,1073.21,1087.705184,1072.445583,1102.964784,1.0,1.0
8,2018-01-03,amazon,Amazon grabbed 4 percent of all US retail sale...,There's no denying that Amazon 's retail busin...,172.23,172.26,173.185404,170.231365,176.139443,0.0,1.0
9,2018-01-03,google,Google discovers 'serious' flaws in Intel and ...,SANTA CLARA Calif. (AP) — Security researcher...,172.23,172.26,173.185404,170.231365,176.139443,0.0,1.0


In [85]:
results = []
for i in range(len(testdata1[testdata1['timeseriesforecast']>0])):
    nextclose = testdata1.loc[i,'nextclose']
    lastclose = testdata1.loc[i,'lastclose']
    timeseriesforecast = testdata1.loc[i,'timeseriesforecast']
    if (nextclose-lastclose) * (timeseriesforecast-lastclose) >= 0:
        results.append(1)
    else:
        results.append(0)
    

In [86]:
np.mean(results)

0.41642228739002934